# **Deploy de un modelo**

En esta notebook vamos a jugar con la creación de un modelo basados en el procesamiento que hicimos en notebooks pasadas y vamos a hacer un mini deploy de dicho modelo.

*Nota*. Para los efectos de esta notebook vamos a usar una estrategia simple de división del dataset en training-test. Se podrían optar por otras opciones u otros esquemas de división. Asimismo, si bien se muestra una posibilidad para la elección del "mejor" modelo mediante una optimización de parámetros, no será utilizada para el modelo a guardar.

### Creación del modelo

Primero, vamos a traernos los datos con los que vamos a estar trabajando. Vamos a seguir utilizando el dataset simple de detección de hate speech, del que nos vamos a quedar con un atributo de tipo texto y la clase numérica (0: No es hate speech, 1: hate speech, 2: offensive speech).

*Nota:* No se preocupen mucho por el procesamiento de texto que hacemos en este ejemplo, ya lo veremos más adelante.

In [ ]:
# Cargamos los datos necesarios
import pandas as pd

url = "https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv"
df = pd.read_csv(url, usecols=['class', 'tweet']) # de todas las columnas que tiene el dataset, nos vamos a quedar solo con el texto y la clase

print(df[:1000]) # limitamos la cantidad de instancias para que no tarde ni el pre-processing ni el training

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size = 0.80,random_state=42) # limitamos el tamaño del training para que no tarde

# recordemos que para entrenar tenemos separar la clase
X_train = train_set.drop("class", axis=1)  
y_train = train_set["class"].copy()

X_test = test_set.drop("class",axis=1) # nos dejamos también preparado el test set
y_test = test_set["class"].copy() # nos dejamos también preparado el test set

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nltk.download('stopwords')

In [ ]:
count_normal = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'))

preprocessor = ColumnTransformer(
    transformers=[
        ('count', count_normal, "tweet")]) # importante definir las columnas sobre las cuales se aplica

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC(probability=True))])

Finalmente, vamos a entrenar el modelo. De acuerdo al modelo que elijamos, puede tardar.

In [ ]:
rf.fit(X_train,y_train)      

In [ ]:
rf.predict(X_test)

### Model selection

El pipeline que definimos también puede ser utilizado en el proceso de selección de modelos. En el siguiente fragmento de código se cicla por diferentes modelos de clasificación provistos por sklearn, para aplicar las transformaciones y luego entrenarlos.

Nota. Hay más clasificadores disponibles para probar.

Nota 2. Puede tardar!!

In [ ]:
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC()
    ]

for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test)) # qué retorna depende del modelo que se usa. En clasificación retorna accuracy promedio

Finalmente, el pipeline que definimos también puede ser utilizado en un grid search para encontrar la mejor combinación de hiper-parámetros.

Para hacer esto, lo primero que hay que hacer es crear una grilla de parámetros para el modelo elegido. Algo importante a notar es que a los nombres de los parámetros hay que agregarles el nombre que le dimos al parámetro que representaba al algoritmo (en este caso de clasificación, al que llamamos ``classifier``).

Luego, creamos el objeto de grid search el cual incluye el pipeline original. Cuando llamemos al método ``fit``, antes de realizar la búsqueda del grid search se aplicarán las transformaciones.

Nota. En este ejemplo se están considerando dos parámetros para el ``RandomForestClassifier``. De acuerdo al clasificador, los parámetros que se podrán optimizar.

Nota 2. Hay múltiples métricas de [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring) que pueden ser consideradas. Ver también la documentación referida a [model evaluation](https://scikit-learn.org/stable/modules/model_evaluation.html).

Nota 3. Puede tardar!!

In [ ]:
from sklearn.model_selection import GridSearchCV

rfcv = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

param_grid = { 
    'classifier__n_estimators': [1, 3, 10],
    'classifier__max_features': ['auto', 'sqrt'],
}

CV = GridSearchCV(rfcv, param_grid, cv=5,
                           scoring='f1_weighted') 
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)

### Salvar/Exportar el modelo

En este caso el modelo lo necesitamos solo acá, pero qué pasaría si nosotros quisieramos llevar este modelo a otro ambiente o simplemente reemplazar otro modelo que teníamos por este. Tenemos que repetir todos los pasos de definición del pipeline y reentrenar? No, no es necesario.

Lo que podemos hacer es persistir el modelo, es decir, guardarlo en un archivo que luego podremos levantar en donde nosotros quisiéramos utilizarlo.

La primera alternativa es usar ```joblib```.

In [ ]:
import joblib

joblib.dump(rf, "hate_speech_detection_model.pkl") 

Luego, para cargarlo:

In [ ]:
loaded_model = joblib.load("hate_speech_detection_model.pkl")
y_pred = loaded_model.predict(X_test)
print(y_pred)

Otra alternativa es usar ```Pickle```.

In [ ]:
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(rf, open(filename, 'wb'))

Luego, para cargarlo:

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X_test)
print(y_pred)

### Deploy del modelo

Para hacer el deploy del modelo, vamos a crear una aplicación de hate speech detection para deployarla como un servicio REST básandonos en el modelo que creamos en los bloques anteriores.

Vamos a usar:

* [Flask](https://github.com/pallets/flask): uno de los micro web frameworks más populares.
* [flask_ngrok](https://pypi.org/project/flask-ngrok/): herramienta que nos permite hacer demos de nuestras apps Flask. Nos permite servir nuestra applicación desde una simple notebook.

Instalamos las dependencias que vamos a necesitar

In [ ]:
!pip install flask
!pip install flask-ngrok

Lo primero que vamos a hacer es dejar accessible nuestro modelo entrenado a nuestra aplicación. 

En este caso, si venimos ejecutando toda la notebook vamos a tener disponible nuestro ``hate_speech_detection_model.pkl``.

Nota. Cada vez que reiniciemos el runtime, deberíamos generar o levantar nuestro modelo de algún lado.

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify
import pandas as pd 
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib

app = Flask(__name__)

run_with_ngrok(app)

@app.route('/')
def home():
  return "<h1>Hate Speech Detector!</h1>"

model = joblib.load("hate_speech_detection_model.pkl") # vamos a levantar nuestro modelo

@app.route('/predict',methods=['GET','POST']) # los tipos de métodos que soportamos
def predict():
  
  df_n = pd.DataFrame({"tweet":[request.args['text']]}) # tomamos el texto que nos pasaron para hacer la predicción

  prediction = model.predict(df_n) # utilizamos el modelo para predecir
  pred_proba = model.predict_proba(df_n) # obtenemos la probabilidad de la predicción --> No está disponible para todos los clasificadores!

  if prediction == 0:
    pred_text = 'Hate'
  elif prediction == 1:
    pred_text = 'Offensive'
  else:
    pred_text = 'Neither'   

  print(request.args['text']) # en la consola de acá imprimimos el texto
  print(pred_proba.dtype) # en la consola de acá imprimimos la probabilidad

  return "<h2>El texto \""+request.args['text']+"\" fue clasificado como: "+pred_text+" con una probabilidad de: "+str(pred_proba[0][prediction])+"</h2>"

app.run()

Y listo! Ahora ya tenemos nuestro detector de hate speech disponible! Si le pasamos un argumento ```text``` con un string nos va a retornar si es o no hate speech! Para pasarle el argumento ``URL_NGROK/predict?text=TEXTO``. Por ejemplo, ``http://b36fd66da979.ngrok.io/predict?text="I hate everyone!!"``

El formato de salida que le dimos no es muy amigable con el usuario si lo que queremos es dejarlo disponible y que otros lo usen. Para eso, vamos a modificar la salida para que nos retorne un json.

In [ ]:
# es el mismo código que antes, solo cambia el return

from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify
import pandas as pd 
from sklearn.externals import joblib
import json

app = Flask(__name__)

run_with_ngrok(app)

@app.route('/')
def home():
  return "<h1>Hate Speech Detector!</h1>"

model = joblib.load("hate_speech_detection_model.pkl") 

@app.route('/predict',methods=['GET','POST'])
def predict():
  
  df_n = pd.DataFrame({"tweet":[request.args['text']]})

  prediction = model.predict(df_n)
  pred_proba = model.predict_proba(df_n)

  if prediction == 0:
    pred_text = 'Hate'
  elif prediction == 1:
    pred_text = 'Offensive'
  else:
    pred_text = 'Neither'  

  output = {'text': request.args['text'], 'prediction': pred_text, 'confidence': str(pred_proba[0][prediction])}

  return json.dumps(output)

app.run()

Si queremos probar de consumirlo como un servicio "normal", podemos ejecutar el siguiente código (en otra notebook, dado que acá estamos ejecutando el server).

In [ ]:
import requests

text = "I think you are not pretty"
base = "COMPLETAR_URL_NGROK"
url = base+'/predict?text='+text

response = requests.post(url)
print(response.content)

Otra posibilidad sería esto mismo deployarlo en algún proveedor Cloud o incluso hacerlo accesible como una imagen de Docker.

